## 1. Poisson's Equation

### Preliminaries

We're looking to solve Poisson's equation for electrostatics (given in dimensionless units, so that $\epsilon_{0} = 1$) on an $(N+1) \times (N+1)$ unit grid:

$$
  \Delta \phi = -\rho
$$

Here, $\rho$ is zero except at $\vec{x} = (0.5 + a/2, 0.25 + a/2)$, where it equals $2\delta(\vec{x})$. Since we're including points along the border, the distance between neighboring grid points is $h = 1/N$. We can express the discretized Laplacian, acting on the $N^2$-vector of potentials on the grid, as:

$$
  \Delta_{i,j} \phi_{j} = \frac{4\phi_{i} - \phi_{i+1} - \phi_{i-1} - \phi_{i+N} - \phi_{i-N}}{h^2}
  = N^2 \Big(4\phi_{i} - \phi_{i+1} - \phi_{i-1} - \phi_{i+N} - \phi_{i-N}\Big)
$$

so that the Laplacian matrix itself can be expressed in terms of Kronecker deltas as

$$
  \Delta_{i,j} = N^2 \Big(4\delta_{i,j} - \delta_{i+1,j} - \delta_{i-1,j} - \delta_{i+N,j} - \delta_{i-N,j}\Big)
$$

which, it's worth noting, is a symmetric matrix. 

We have inhomogenous boundary conditions ($\phi = 10$) for the rectangle centered at (0.625, 0.75) and homogenous boundary conditions along the perimeter of the grid. Since these values of $\phi$ are fixed, they cannot figure into the $\Delta\phi=-\rho$ problem, in the sense that $\rho$ cannot be fixed if $\phi$ is held fixed (the equivalent physical analogy is the surface of a conductor in electrostatics). So for any _fixed_ $\phi_{i}$, we can set $\rho_{i} = 0 \implies \Delta_{i,j} = 0 \ \ \forall \ \ j$. In other words, for each $i$ such that $\phi_{i}$ is held constant, we set the $i^{th}$ row of $\Delta$ equal to zero. This assures us that $\phi$ only depends on $\rho$ through the boundary conditions.

Since the conjugate gradient method depends on the symmetry of a matrix, we'll have to deal with the nonzero terms in the $i^{th}$ _column_ of $\Delta$ before zeroing the $i^{th}$ row, which are:

$$
  \Delta_{j,i} = N^2 \Big(4\delta_{i,j} - \delta_{i,j+1} - \delta_{i,j-1} - \delta_{i,j+N} - \delta_{i,j-N}\Big).
$$

Since these products are to be multiplied by the constant $\phi$ values, we can move them over to the right hand side of Poisson's equation. First, define $\beta_{i}$ so that it is one when $\phi_{i}$ is a boundary condition and zero otherwise. Then we split $\phi$ into fixed and free components

$$
  \vec{\phi} = \vec{\phi}_{free} + \vec{\phi}_{boundary}
$$

where

$$
  \vec{\phi}_{boundary} \equiv \sum_{i} \vec{\phi}^{T} \hat{e}_{i} \beta_{i}
$$

and $\vec{\phi}_{free} = \vec{\phi} - \vec{\phi}_{boundary}$. We can similarly split $\Delta$ by its columns,

$$
  \Delta = \Delta_{free} + \Delta_{boundary},
$$

so that the columns corresponding to boundary values in $\Delta_{free}$ are zero, i.e., independent of fixed quantities, implying $\Delta_{free} \vec{\phi}_{boundary} = \Delta_{boundary} \vec{\phi}_{free} = 0$. Then Poisson's equation becomes

$$
  \Delta_{free} \phi_{free} = -(\rho +\Delta_{boundary} \phi_{boundary}) \equiv -\rho_{boundary}
$$

where $\rho_{boundary}$ now contains the potential boundary conditions as well as the charge density. The left hand side is now free of fixed quantities; we can zero the boundary rows on both sides by taking the elemental product with the sum of free identity vectors

$$
  e_{free} \equiv \sum_{free\ i} \hat{e}_{i} 
$$

so that the equation becomes

$$
  \Delta_{eff} \phi_{free} \equiv \Delta_{free} \phi_{free} \ .\times\ e_{free} 
  = -\rho_{boundary} \ .\times\ e_{free} \equiv -\rho_{eff}.
$$

or, more briefly,

$$
  \Delta_{eff} \phi_{free} = -\rho_{eff}.
$$

Once we've solved for $\phi_{free}$ using the conjugate gradient method, we can reconstitute the original equation and, if necessary, calculate the values of $\rho$ that allowed for the boundary conditions.

I also partially implemented a feature that splits off the free parameters into their own Poisson equation, but haven't ended up finishing it. Since $\Delta_{eff}$, $\phi_{free}$, and $\rho_{eff}$ have zeros in every row (and column) associated with the boundary conditions, we can define minor matrices for each with those rows/columns removed entirely,

$$
  \Delta^\prime \phi^\prime = -\rho^\prime,
$$

along with an integer array, `freetofull`, with the same length as $\rho^\prime$ and which maps from its own indices to the corresponding indices in the original, full $\phi$ and $\rho$ vectors. This allows us to reduce the size of the problem and speed up the algorithm by making a clean distinction between the components of the potential representing boundary conditions and the components whose values we can actually optimize.

### Implementation

My Laplacian.jl module implements, in order of increasing abstraction:

* ($\delta$ or `kronecker`), __a Kronecker delta matrix function__ that takes arbitrary anonymous functions $f_i$ and $f_j$ (of row and column number, respectively) as arguments, and generates a matrix of arbitrary size whose elements are $\delta_{f_i,f_j}$, allowing for convenient translation of the above formulas
* (`lap`), __a Laplacian matrix function__ that uses `kronecker` along with the above formula for $\Delta$ to produce a Laplacian operator matrix for a square grid, given a particular side length and number of subdivisions in each direction.
* (`findfree`), __a function that produces an array mapping its indices to the free indices of the full matrix__ and which takes as its arguments the size of the full matrix ($N^2$) and a list of indices whose values are fixed as boundary conditions
* (`opsplit`), __a function for splitting $\Delta$ into free and boundary parts__ that takes an $N \times N$ Laplacian matrix (or, in general, a matrix) and an integer array giving the indices of boundary conditions, and returns the tuple ($\Delta_{free}$, $\Delta_{boundary}$) as output.
* (`rhoeff`), __a function for calculating $\rho_{eff}$__ that takes $\rho$, $\Delta_{boundary}$, $\phi_{boundary}$, and an array of the boundary condition indices as input.

__Going forward, I will drop the subscripts in the above version of Poisson's equation unless otherwise specified for the sake of simplicity.__ 